<a href="https://colab.research.google.com/github/ozguozkan/DI725_Assign2/blob/main/DI725_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

connecting to the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

importing necessary libs

In [ ]:
import os
import json
from __future__ import annotations
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import random_split
import wandb
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm

wandb login

In [ ]:
!pip install -q wandb

In [ ]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ozgukan to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

getting the detr_resnet50 model

In [ ]:
!pip install -q transformers

from transformers import DetrForObjectDetection, DetrImageProcessor

my paths in colab

In [ ]:
path = '/content/drive/MyDrive/auair2019'
img_path = os.path.join(path, 'images')
annot_path = os.path.join(path, 'annotations.json')

reading the annotations

In [ ]:
with open(annot_path, 'r') as f:
  annotations = json.load(f)

print(json.dumps(annotations['annotations'][0], indent=2)) #seeing the first images info


{
  "image_name": "frame_20190829091111_x_0001973.jpg",
  "image_width:": 1920.0,
  "image_height": 1080.0,
  "platform": "Parrot Bebop 2",
  "time": {
    "year": 2019,
    "month": 8,
    "day": 29,
    "hour": 9,
    "min": 11,
    "sec": 11,
    "ms": 394400.0
  },
  "longtitude": 10.18798203255313,
  "latitude": 56.20630134795274,
  "altitude": 19921.6,
  "linear_x": 0.03130074199289083,
  "linear_y": 0.028357808757573367,
  "linear_z": 0.0744575835764408,
  "angle_phi": -0.06713105738162994,
  "angle_theta": 0.06894744634628296,
  "angle_psi": 1.1161083340644837,
  "bbox": [
    {
      "top": 163,
      "left": 1098,
      "height": 185,
      "width": 420,
      "class": 1
    },
    {
      "top": 421,
      "left": 1128,
      "height": 176,
      "width": 393,
      "class": 1
    },
    {
      "top": 927,
      "left": 1703,
      "height": 153,
      "width": 183,
      "class": 0
    }
  ]
}


In [ ]:
print("Toplam annotation sayısı:", len(dataset.annotations))

Toplam annotation sayısı: 32823


i need to fix image_width: problem

i need to change the annotaiton format into  (x min, y min, x max, y max) and i ned the match each annotation with its image

In [ ]:
image_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
class auairdata(Dataset):
  def __init__(self, annot_path, img_dir, transform = None):
    with open(annot_path, 'r') as f:
      all_data = json.load(f)
      self.annotations = all_data["annotations"]

    self.img_dir = img_dir
    self.transform = transform

    self.converted_annotations = []

    for idx, annot in enumerate(self.annotations):
      if "image_width:" in annot:
        annot["image_width"] = annot.pop("image_width:")



    #to make my data compatible with the hw i change the variables
    #pytorch expects the data as x_min, y_max etc.
      for bbox in annot["bbox"]:
        x = bbox["left"]
        y = bbox["top"]
        w = bbox["width"]
        h = bbox["height"]
        cls = bbox["class"]

        self.converted_annotations.append({
            "image_id": idx,
            "bbox": [x, y, w, h],
            "category_id": cls,
            "area": w * h,
            "iscrowd": 0
        })

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, idx): #the annotation and the image has been matched
    annot = self.annotations[idx]
    img_path = os.path.join(self.img_dir, annot["image_name"])
    image = Image.open(img_path).convert("RGB")

    coco_labels = [ann for ann in self.converted_annotations if ann ["image_id"] == idx]


    mytargets = { #because torch.vision expects the boxes and labels in this format
        # 'image_id': torch.tensor([idx]),
        'image_id': idx,
        'annotations': coco_labels
    }

    encode = image_processor(image, annotations=mytargets, return_tensors="pt")

    return {
        "pixel_values": encode["pixel_values"].squeeze(0),
        "labels": encode["labels"][0]
    }


#preparig the dataset

dataset = auairdata(
    annot_path='/content/drive/MyDrive/auair2019/annotations.json',
    img_dir='/content/drive/MyDrive/auair2019/images',
    transform=None
)

def collate_fn(batch): #i used collate_fn for getting examples from dataloader and to send them into a batch
  pixel_values = [item["pixel_values"] for item in batch]
  labels = [item["labels"] for item in batch]

  return {
      "pixel_values": torch.stack(pixel_values),
      "labels": labels

  }

dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

#saving as coco
save_path = '/content/drive/MyDrive/auair2019/converted_annotations.json'
with open(save_path, 'w') as f:
  json.dump({"annotations": dataset.converted_annotations}, f, indent=2)

print(f"json saved: {save_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

json saved: /content/drive/MyDrive/auair2019/converted_annotations.json


In [ ]:
import json

#img id img name match
with open('/content/drive/MyDrive/auair2019/annotations.json', 'r') as f:
    auair_data = json.load(f)

image_id_to_name = {}
for idx, annot in enumerate(auair_data["annotations"]):
    if "image_name" in annot:
        image_id_to_name[idx] = annot["image_name"]

with open('/content/drive/MyDrive/auair2019/converted_annotations.json', 'r') as f:
    converted = json.load(f)

#add imgname evey annot
for ann in converted["annotations"]:
    img_id = ann["image_id"]
    ann["image_name"] = image_id_to_name.get(img_id, f"frame_{img_id:07d}.jpg")  # fallback olarak üret

save_path = '/content/drive/MyDrive/auair2019/converted_annotations_with_names.json'
with open(save_path, 'w') as f:
    json.dump(converted, f, indent=2)

print(f"new json saved: {save_path}")


new json saved: /content/drive/MyDrive/auair2019/converted_annotations_with_names.json


one time only

In [ ]:
class CocoStyleDataset(Dataset):
    def __init__(self, annotation_path, img_dir, transform=None):
        with open(annotation_path, 'r') as f:
            all_data = json.load(f)
            self.annotations = all_data["annotations"]

        self.img_dir = img_dir
        self.transform = transform

        # if there is image name group them
        self.image_to_annots = {}
        for ann in self.annotations:
            img_name = ann["image_name"]
            self.image_to_annots.setdefault(img_name, []).append(ann)

        self.image_names = sorted(self.image_to_annots.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        img_path = os.path.join(self.img_dir, image_name)
        image = Image.open(img_path).convert("RGB")

        annots = self.image_to_annots[image_name]

        mytargets = {
            "image_id": idx,
            "annotations": annots
        }

        encode = image_processor(image, annotations=mytargets, return_tensors="pt")

        return {
            "pixel_values": encode["pixel_values"].squeeze(0),
            "labels": encode["labels"][0]
        }


In [ ]:
dataset = CocoStyleDataset(
    annotation_path='/content/drive/MyDrive/auair2019/converted_annotations_with_names.json',
    img_dir='/content/drive/MyDrive/auair2019/images'
)

In [ ]:
category_id_to_name = {
    0: "human",
    1: "car",
    2: "truck",
    3: "van",
    4: "motorbike",
    5: "bicycle",
    6: "bus",
    7: "trailer"

}

In [ ]:
# get the ids
category_ids = [ann["category_id"] for ann in dataset.converted_annotations]
category_counts = Counter(category_ids)

print("category dist:\n")
for cat_id in sorted(category_counts):
    name = category_id_to_name.get(cat_id, "Unknown")
    count = category_counts[cat_id]
    print(f"{name} (category {cat_id}): {count} sample")

AttributeError: 'CocoStyleDataset' object has no attribute 'converted_annotations'

checking how many classes are there in the auair datasaet

In [ ]:
with open('/content/drive/MyDrive/auair2019/annotations.json', 'r') as f:
    data = json.load(f)

annotations = data["annotations"]  # my annot list is here

class_ids = set()

for ann in annotations:
    for bbox in ann["bbox"]:
        class_ids.add(bbox["class"])

print(f"total class num: {len(class_ids)}")
print(f"classes: {sorted(class_ids)}")


total class num: 8
classes: [0, 1, 2, 3, 4, 5, 6, 7]


In [ ]:
with open('/content/drive/MyDrive/auair2019/annotations.json', 'r') as f:
    data = json.load(f)

print(f"Top level keys: {data.keys()}")

Top level keys: dict_keys(['info', 'licenses', 'categories', 'annotations'])


In [ ]:
with open('/content/drive/MyDrive/auair2019/annotations.json', 'r') as f:
    data = json.load(f)

print("first annot:")
print(data["annotations"][0])


first annot:
{'image_name': 'frame_20190829091111_x_0001973.jpg', 'image_width:': 1920.0, 'image_height': 1080.0, 'platform': 'Parrot Bebop 2', 'time': {'year': 2019, 'month': 8, 'day': 29, 'hour': 9, 'min': 11, 'sec': 11, 'ms': 394400.0}, 'longtitude': 10.18798203255313, 'latitude': 56.20630134795274, 'altitude': 19921.6, 'linear_x': 0.03130074199289083, 'linear_y': 0.028357808757573367, 'linear_z': 0.0744575835764408, 'angle_phi': -0.06713105738162994, 'angle_theta': 0.06894744634628296, 'angle_psi': 1.1161083340644837, 'bbox': [{'top': 163, 'left': 1098, 'height': 185, 'width': 420, 'class': 1}, {'top': 421, 'left': 1128, 'height': 176, 'width': 393, 'class': 1}, {'top': 927, 'left': 1703, 'height': 153, 'width': 183, 'class': 0}]}


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#as we seen before in auair dataset there are 8 classes and the backgorund so 9 classes
#i need to adjust the class number for the detr model
#the detr also wants tje background thats  why i add it

model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels=9,  # class num
    ignore_mismatched_sizes=True
)

model.to(device)
model.train()


config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

splitting the dataset

In [ ]:
#%70 train, %15 val, %15 test
ful_len = len(dataset)
train_len = int(0.7*ful_len)
val_len = int(0.15*ful_len)
test_len = ful_len - train_len - val_len

print(f"train: {train_len}, val: {val_len}, test:{test_len}")

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_len, val_len, test_len],
    generator=torch.Generator().manual_seed(42)
)

#i add num workers an pin memory bc 4 epochs lasted 11.30 hours
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)

train: 22976, val: 4923, test:4924


In [ ]:
wandb.init(project = "DI725-Assignment2", name = "detr-finetune", config = {
    "epochs": 5,
    "batch_size": 8,
    "lr": 5e-5
})

model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels = 9,
    ignore_mismatched_sizes = True
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5)

train_losses = []
val_losses = []

def evaluate(model, dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in dataloader:
            pixel_values = batch["pixel_values"].to(device)
            labels = [{k: v.to(device) for k, v in l.items()} for l in batch["labels"]]
            outputs = model(pixel_values=pixel_values, labels=labels)
            total_loss += outputs.loss.item()
    return total_loss / len(dataloader)


epochs = wandb.config["epochs"]
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    train_bar = tqdm(train_loader, desc=f"epoch {epoch+1}/{epochs}", leave=False)
    for batch in train_bar:
        pixel_values = batch["pixel_values"].to(device)
        labels = [{k: v.to(device) for k, v in l.items()} for l in batch["labels"]]

        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_bar.set_postfix({"Loss": loss.item()})

    train_loss = running_loss / len(train_loader)
    val_loss = evaluate(model, val_loader)
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    # only on last epoch
    if epoch == epochs - 1:
        test_loss = evaluate(model, test_loader)
        wandb.log({"test_loss": test_loss})
        print(f"[Epoch {epoch+1}] test Loss: {test_loss:.4f}")
    else:
        test_loss = None

    print(f"[Epoch {epoch+1}] train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
    }, step=epoch + 1)

#plot
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
plt.plot(range(1, len(val_losses)+1), val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Validation Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#wandb’ye de loss grafiği gönder
wandb.log({
    "train_val_loss_curve": wandb.plot.line_series(
        xs=list(range(1, len(train_losses)+1)),
        ys=[train_losses, val_losses],
        keys=["Train", "Val"],
        title="Loss Over Epochs",
        xname="Epoch"
    )
})

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

KeyboardInterrupt: 